In [1]:
# import
import pandas as pd
import folium
import requests
from urllib.parse import quote
import json

In [4]:
road_api = 'devU01TX0FVVEgyMDIyMTIyOTE2NTgxMTExMzM4NzA='
kakao_api = 'c6fb0ac29226c99411f33661deb221d4'

In [14]:
my_gohyang=['영등포구청', '구로구청', '서초구청', '양천구청']

In [26]:
my_go_addr=[]
for i in my_gohyang:
    base_url = 'https://business.juso.go.kr/addrlink/addrLinkApi.do'
    params1 = f'currentPage=1&countPerPage=10&confmKey={road_api}'
    params2 = f'keyword={quote(i)}&resultType=json'
    url = f'{base_url}?{params1}&{params2}'
    result = requests.get(url)
    res = json.loads(result.text)
    my_go_addr.append(res['results']['juso'][0]['roadAddr'])
my_go_addr

['서울특별시 영등포구 당산로 지하121(당산동3가)',
 '서울특별시 구로구 가마산로 245(구로동)',
 '서울특별시 서초구 남부순환로 2584(서초동)',
 '서울특별시 양천구 목동동로 105(신정동)']

In [27]:
df=pd.DataFrame({'장소':my_gohyang,'도로명주소':my_go_addr})
df

,장소,도로명주소
0,영등포구청,서울특별시 영등포구 당산로 지하121(당산동3가)
1,구로구청,서울특별시 구로구 가마산로 245(구로동)
2,서초구청,서울특별시 서초구 남부순환로 2584(서초동)
3,양천구청,서울특별시 양천구 목동동로 105(신정동)


In [35]:
uidos=[]
gyeongdoes=[]
for i in range(4):
    addr = df.iloc[i,1]
    search_url = 'https://dapi.kakao.com/v2/local/search/address.json'
    url2 = f'{search_url}?query={quote(addr)}'
    result2 = requests.get(url2,
                     headers={'Authorization': f'KakaoAK {kakao_api}'}).json()
    uidos.append(result2['documents'][0]['address']['y'])
    gyeongdoes.append(result2['documents'][0]['address']['x'])
print(uidos)
print(gyeongdoes)

['37.5256996614804', '37.4955054632154', '37.4836248649455', '37.5170753784215']
['126.896626898379', '126.888292375229', '127.032683002019', '126.866542541936']


In [37]:
df['위도']=uidos
df['경도']=gyeongdoes
df

,장소,도로명주소,위도,경도
0,영등포구청,서울특별시 영등포구 당산로 지하121(당산동3가),37.5256996614804,126.896626898379
1,구로구청,서울특별시 구로구 가마산로 245(구로동),37.4955054632154,126.888292375229
2,서초구청,서울특별시 서초구 남부순환로 2584(서초동),37.4836248649455,127.032683002019
3,양천구청,서울특별시 양천구 목동동로 105(신정동),37.5170753784215,126.866542541936


In [38]:
seoul_map = folium.Map(location=[37.55,126.96],zoom_start=12)
for name, lat, lng in zip(df.장소,df.위도,df.경도):
    folium.Marker([lat,lng],popup=name).add_to(seoul_map)
seoul_map